In [13]:
import numpy as np
import random
import pickle
import time

In [638]:
learning_rate=0.05 #alpha
decay_gamma=0.7 #gamma
exp_rate=0.3 #epsilon
starting_player='dummy'
list_of_winners=[]
policy={}
number_of_pits=4
number_of_stones=4

In [639]:
class Imkalah:
    def __init__(self, pits, pebbles):
        self.starting_player=starting_player
        self.player=starting_player
        self.pits=pits
        self.pebbles=pebbles
        self.board=[pebbles]*pits+[0]
        self.number_of_pits=len(self.board)
        self.lr=learning_rate
        self.gamma=decay_gamma
        self.exp_rate=exp_rate
        self.Q_table={}
        self.game_over=False
        self.states=[]
    def choose_random_pit(self):
        """This function chooses a random pit to choose.
        Returns an index for a non-empty game pit"""
        return random.choice(list(np.nonzero(self.board[:-1])[0]))
    def choose_smart_pit(self):
        """This function chooses the most optimal non-empty game pit according to the Q_table.
        Returns the index for the best choice of non-empty game pit"""
        current_board=self.board.copy()
        boards_to_evaluate_in_Q_table=[]
        possible_moves_1=list(np.nonzero(current_board[:-1])[0])
        boards_to_check=[]
        for pit_to_choose in possible_moves_1:
            #print('ER DET HER DER ER FEJL', pit_to_choose, possible_moves_1, current_board.copy())
            turn, _, test_board=self.test_move(pit_to_choose, current_board.copy())
            #print('ER DET HER DER ER FEJL2', pit_to_choose, possible_moves_1, test_board)
            if turn=='over':
                boards_to_evaluate_in_Q_table.append((pit_to_choose,test_board))
            else:
                boards_to_check.append(test_board.copy())
                while len(boards_to_check)>0:
                    current_board_1=boards_to_check.pop(0)
                    done=False
                    possible_moves=list(np.nonzero(current_board_1[:-1])[0])
                    for move in possible_moves:
                        #print('ELLER ER DET HER, DER ER FEJL', move, possible_moves, current_board_1.copy())
                        turn, _, test_board=self.test_move(move, current_board_1.copy())
                        #print('ELLER ER DET HER, DER ER FEJL2', move, possible_moves, test_board)
                        if turn=='over':
                            boards_to_evaluate_in_Q_table.append((pit_to_choose,test_board))
                        else:
                            boards_to_check.append(test_board.copy())
        value_max=-999
        best_index=None
        for next_board in boards_to_evaluate_in_Q_table:
            value = 0 if self.Q_table.get(str(next_board[1])) is None else self.Q_table.get(str(next_board[1]))
            if value >= value_max:
                value_max = value
                best_index=next_board[0]
        return best_index
    def possible_next_states(self, init_board):
        """This function finds all possible board states achieveable from a given state.
        Input is a board for which to find achieveable board states from
        Returns a list of board states achieveable from the input board."""
        if init_board[-1]==self.pebbles*self.pits:
            return None
        list_of_board_states=[]
        boards_to_check=[init_board.copy()]
        while len(boards_to_check)>0:
            current_board=boards_to_check.pop(0)
            possible_moves=list(np.nonzero(current_board[:-1])[0])
            for pit_to_choose in possible_moves:
                test_board=current_board.copy()
                turn, game_over, test_board = self.test_move(pit_to_choose, test_board)
                if turn=='over' or game_over==True:
                    list_of_board_states.append(test_board)
                else:
                    boards_to_check.append(test_board)
        return [list(x) for x in set(tuple(subliste) for subliste in list_of_board_states)]     
    def test_move(self, pit_index, test_board):
        """This function is for testing the outcome of choosing a certain non-empty game pit from a certain game state
        Input is the index of the non-empty game pit and the board state to test from.
        Returns:
            turn: is either ongoing or over, depending on whether the move ends the turn or requires another move
            game_over: is true if the move results in all pebbles being in the target pit.
            test_board: the board state after the move is done"""
        turn='ongoing'
        #print('TESTBOARD', test_board, 'INDEX', pit_index)
        pit_to_choose=pit_index
        game_over=False
        while turn=='ongoing' and game_over==False:
            pebbles_on_hand=test_board[pit_to_choose]
            #print('PEBBLES ON HAND', pebbles_on_hand)
            test_board[pit_to_choose]=0
            for i in range(pebbles_on_hand):
                pit_to_add_to=(pit_to_choose+1+i)%(len(test_board))
                test_board[pit_to_add_to]+=1
                #print('board while performing move', test_board)
            if pit_to_add_to==len(test_board)-1: #if last stone is placed in j0
                if test_board[-1]==sum(test_board): #if it is the last stone in the game
                    game_over=True
                    turn='over'
                else:
                    return turn, game_over, test_board
            elif test_board[pit_to_add_to]==1:#if last stone is placed in empty pit
                turn='over'
            else:
                pit_to_choose=pit_to_add_to #last stone is not placed in empty pit, so continue
        return turn, game_over, test_board
    def perform_move(self, pit_index):
        """Function to perform a move to the actual game board.
        input: pit_index is the index(0-indexed) of the pit in the board which the player
                should take the pebbles from
        Returns whether or not the turn is over by performing the move."""
        turn='ongoing'
        pit_to_choose=pit_index
        while turn=='ongoing' and self.game_over==False:
            pebbles_on_hand=self.board[pit_to_choose]
            self.board[pit_to_choose]=0
            for i in range(pebbles_on_hand):
                pit_to_add_to=(pit_to_choose+1+i)%self.number_of_pits
                self.board[pit_to_add_to]+=1
            if pit_to_add_to==len(self.board)-1: #if last stone is placed in j0
                if self.board[-1]==self.pebbles*self.pits: #if it is the last stone in the game
                    self.game_over=True
                    turn='over'
                else:
                    return turn
            elif self.board[pit_to_add_to]==1:#if last stone is placed in empty pit
                turn='over'
            else:
                pit_to_choose=pit_to_add_to #last stone is not placed in empty pit, so continue
        return turn
    def change_player(self):
        """Function for changing current player.
        Returns None"""
        if self.player=='agent':
            self.player='dummy'
        else:
            self.player='agent'
    def reset(self):
        """Function for resetting the game.
        Returns None"""
        self.game_over=False
        winner=None
        self.player=starting_player
        self.j0=0
        self.board=[self.pebbles]*self.pits+[0]
        self.states=[]
    def feedReward_(self, reward):
        """
        THIS REWARD FUNCTION DOES NOT CONSIDER POSSIBLE NEXT STATES
        """
        for st in reversed(self.states):  # goes through all saved board states of this game
            if self.Q_table.get(str(st)) is None:  # if it's not already in the dictionary (of board states of ALL games)
                self.Q_table[str(st)] = 0  # initialise a value for the state
            self.Q_table[str(st)] += self.lr * (
                        self.gamma * reward - self.Q_table[str(st)])  # update weight for each board state
    def feedReward(self, reward):
        """Function for updating the Q_table.
        Input is the reward/penalty to provide board states from a game.
        Returns None"""
        #print('reversed_self.states', self.states)
        for st in reversed(self.states):  # goes through all saved board states of this game
            #print('STATE', st)
            temp=-1000
            possible_next_st=self.possible_next_states(st)
            #print(st, possible_next_st, 'hej')
            if possible_next_st is not None:
                for i in possible_next_st:
                    #if self.Q_table.get(str(i)) is not None:
                        #print(i, self.Q_table.get(str(i)), 'halløj')
                    if self.Q_table.get(str(i)) is not None and self.Q_table.get(str(i))>temp:
                        temp=self.Q_table.get(str(i))
                if self.Q_table.get(str(st)) is None:
                    self.Q_table[str(st)] = 0  # initialise a value for the state
                self.Q_table[str(st)] += self.lr * (reward -self.gamma*temp- self.Q_table[str(st)])
            else:
                if self.Q_table.get(str(st)) is None:  # if it's not already in the dictionary (of board states of ALL games)
                    self.Q_table[str(st)] = 0  # initialise a value for the state
                self.Q_table[str(st)] += self.lr * (
                        self.gamma * reward - self.Q_table[str(st)])  # update weight for each board state
    def training_game(self, rounds=1000):
        """Function for training the agent.
        Returns None"""
        for i in range(rounds):
            while not self.game_over:
                if self.player=='dummy':
                    turn='ongoing'
                    while turn=='ongoing':
                        move= self.choose_random_pit()
                        turn=self.perform_move(move)
                else:
                    if np.random.uniform(0,1)<= self.exp_rate:
                        turn='ongoing'
                        while turn=='ongoing':
                            move= self.choose_random_pit()
                            turn=self.perform_move(move)
                            self.states.append(self.board.copy())
                    else:
                        turn='ongoing'
                        while turn=='ongoing':
                            move= self.choose_smart_pit()
                            turn=self.perform_move(move)
                            self.states.append(self.board.copy())
                if self.game_over:
                    winner=self.player
                    list_of_winners.append(winner)
                    if winner=='agent':
                        self.feedReward(5)
                    else:
                        self.feedReward(-1)
                    self.reset()
                    break
                else:
                    self.change_player()
    def testing_game(self, rounds=1000):
        """Function for testing the agent using the Q_table obtained from training.
        Returns None"""
        for i in range(rounds):
            while not self.game_over:
                if self.player=='dummy':
                    turn='ongoing'
                    while turn=='ongoing':
                        move= self.choose_random_pit()
                        turn=self.perform_move(move)
                else:
                        turn='ongoing'
                        while turn=='ongoing':
                            move= self.choose_smart_pit()
                            turn=self.perform_move(move)
                            self.states.append(self.board)
                if self.game_over:
                    winner=self.player
                    list_of_winners.append(winner)
                    self.reset()
                    break
                else:
                    self.change_player()

In [640]:
play=Imkalah(number_of_pits,number_of_stones)

In [641]:
rounds=1000
list_of_winners=[]
start_time=time.time()
while list_of_winners.count('agent')*100/rounds <100:
    list_of_winners = []
    play.training_game(rounds)
    #print('agent wins', list_of_winners.count('agent')*100/rounds, 'percent of the training games')
    list_of_winners=[]
    play.testing_game(rounds)
    print('agent wins', list_of_winners.count('agent')*100/rounds, 'percent of the test games', len(play.Q_table))
end_time=time.time()
print('it took', round(end_time-start_time,3), 'seconds to finish')

agent wins 99.5 percent of the test games 312
agent wins 99.9 percent of the test games 343
agent wins 99.7 percent of the test games 354
agent wins 99.6 percent of the test games 369
agent wins 99.7 percent of the test games 375
agent wins 99.9 percent of the test games 377
agent wins 100.0 percent of the test games 378
it took 18.458 seconds to finish


In [642]:
list_of_winners=[]
play.testing_game(5000)
print('agent wins', list_of_winners.count('agent')*100/5000, 'percent of the test games', len(play.Q_table))

agent wins 100.0 percent of the test games 378


In [682]:
play.board=[4,4,4,4,0]

In [683]:
boards_to_check=play.possible_next_states(play.board)
boards_to_check

[[1, 1, 6, 6, 2],
 [1, 6, 1, 6, 2],
 [6, 1, 1, 6, 2],
 [1, 6, 6, 1, 2],
 [6, 6, 1, 1, 2]]

In [684]:
for i in boards_to_check:
    print('START', i)
    play.board=i
    move=play.choose_smart_pit()
    print(move)
    turn=play.perform_move(move)
    board=play.board
    print(board)
    while turn=='ongoing':
        move=play.choose_smart_pit()
        print(move)
        turn=play.perform_move(move)
        board=play.board
        print(board)
    print(board)

START [1, 1, 6, 6, 2]
3
[2, 2, 7, 1, 4]
2
[3, 3, 1, 3, 6]
1
[3, 0, 2, 4, 7]
2
[3, 0, 0, 5, 8]
0
[1, 2, 2, 1, 10]
3
[1, 2, 2, 0, 11]
2
[1, 2, 0, 1, 12]
1
[0, 1, 0, 1, 14]
3
[0, 1, 0, 0, 15]
1
[0, 0, 1, 0, 15]
[0, 0, 1, 0, 15]
START [1, 6, 1, 6, 2]
0
[3, 0, 5, 2, 6]
3
[0, 1, 6, 1, 8]
3
[0, 1, 6, 0, 9]
2
[0, 3, 0, 1, 12]
3
[0, 3, 0, 0, 13]
1
[0, 0, 1, 1, 14]
2
[1, 0, 0, 0, 15]
[1, 0, 0, 0, 15]
START [6, 1, 1, 6, 2]
3
[7, 2, 2, 1, 4]
2
[7, 2, 0, 2, 5]
0
[1, 4, 0, 4, 7]
1
[0, 1, 0, 6, 9]
3
[1, 2, 1, 1, 11]
3
[1, 2, 1, 0, 12]
0
[0, 0, 2, 1, 13]
2
[0, 0, 0, 2, 14]
3
[1, 0, 0, 0, 15]
[1, 0, 0, 0, 15]
START [1, 6, 6, 1, 2]
3
[1, 6, 6, 0, 3]
1
[0, 3, 2, 4, 7]
2
[0, 3, 0, 5, 8]
1
[0, 0, 1, 6, 9]
3
[1, 1, 2, 1, 11]
0
[1, 0, 3, 0, 12]
[1, 0, 3, 0, 12]
START [6, 6, 1, 1, 2]
2
[1, 8, 0, 2, 5]
1
[2, 1, 2, 4, 7]
1
[1, 2, 2, 1, 10]
3
[1, 2, 2, 0, 11]
2
[1, 2, 0, 1, 12]
1
[0, 1, 0, 1, 14]
3
[0, 1, 0, 0, 15]
1
[0, 0, 1, 0, 15]
[0, 0, 1, 0, 15]


In [673]:
move=play.choose_smart_pit()
move

1

In [674]:
print(play.board)
print(play.perform_move(move))
print(play.board)


[0, 1, 0, 0, 15]
over
[0, 0, 1, 0, 15]


In [602]:
play.board

[1, 0, 0, 8]

In [540]:
print(play.choose_smart_pit())
play.perform_move(play.choose_smart_pit())
print(play.board)

0
[1, 0, 3]


In [591]:
import ast

my_dict=play.Q_table
sorted_dict = sorted(my_dict.items(), key=lambda x: x[1], reverse=True)
top_keys = [x[0] for x in sorted_dict]

best_boards=top_keys
for i in best_boards:
    print( i,round(my_dict[str(i)],3))

[0, 1, 0, 8] 5.057
[0, 0, 0, 9] 3.5
[1, 3, 0, 5] 3.411
[0, 1, 2, 6] 3.222
[0, 0, 6, 3] 3.105
[0, 3, 1, 5] 2.888
[0, 2, 0, 7] 2.55
[0, 1, 1, 7] 2.527
[2, 0, 0, 7] 2.474
[1, 5, 0, 3] 2.352
[0, 2, 1, 6] 2.327
[2, 2, 0, 5] 2.267
[1, 1, 3, 4] 1.85
[0, 0, 3, 6] 1.778
[0, 2, 2, 5] 1.679
[2, 0, 2, 5] 1.544
[2, 2, 1, 4] 1.543
[1, 0, 1, 7] 1.499
[5, 1, 0, 3] 1.481
[2, 0, 1, 6] 1.464
[1, 0, 0, 8] 1.425
[0, 0, 4, 5] 0.952
[5, 0, 1, 3] 0.917
[2, 1, 0, 6] 0.784
[1, 1, 2, 5] 0.316
[2, 1, 1, 5] 0.223
[0, 0, 1, 8] -0.018
[1, 1, 1, 6] -0.313
[1, 1, 0, 7] -0.441
[1, 2, 1, 5] -0.725
[1, 2, 0, 6] -1.448
[0, 0, 2, 7] -2.444
[3, 0, 1, 5] -3.092
[3, 0, 0, 6] -5.865


In [210]:
play.possible_next_states([5, 1, 1, 2])

[[1, 2, 1, 5], [0, 1, 1, 7], [0, 3, 1, 5], [5, 0, 1, 3], [0, 1, 0, 8]]

In [211]:
print(play.Q_table['[0, 1, 0, 8]'])

KeyError: '[0, 1, 0, 8]'